## New branch

In [ ]:
using Revise, LazySets, StaticArrays, BenchmarkTools, Test
using LazySets: isapproxzero

In [ ]:
a = BallInf([1.0, 2.0], 2.0)
b = BallInf(SA[1.0, 2.0], 2.0);

@btime convert(Zonotope, $a)
@btime convert(Zonotope, $b)
@btime LazySets._convert_2D_static(Zonotope, $b)

h = rand(Hyperrectangle)
hs = Hyperrectangle(SVector{2,Float64}(h.center), SVector{2,Float64}(h.radius))

@btime convert(Zonotope, $h)
@btime convert(Zonotope, $hs)
@btime LazySets._convert_2D_static(Zonotope, $hs)

```julia
  39.723 ns (1 allocation: 112 bytes)
  68.085 ns (3 allocations: 144 bytes)
  0.016 ns (0 allocations: 0 bytes)
  41.247 ns (1 allocation: 112 bytes)
  66.996 ns (3 allocations: 144 bytes)
  0.016 ns (0 allocations: 0 bytes)
```

## Master

In [ ]:
using Revise, LazySets, StaticArrays, BenchmarkTools, Test
using LazySets: isapproxzero

In [ ]:
a = BallInf([1.0, 2.0], 2.0)
b = BallInf(SA[1.0, 2.0], 2.0);

@btime convert(Zonotope, $a)
@btime convert(Zonotope, $b)

h = rand(Hyperrectangle)
hs = Hyperrectangle(SVector{2,Float64}(h.center), SVector{2,Float64}(h.radius))

@btime convert(Zonotope, $h)
@btime convert(Zonotope, $hs)

```
  103.428 ns (3 allocations: 240 bytes)
  97.872 ns (3 allocations: 240 bytes)
  104.838 ns (3 allocations: 240 bytes)
  98.872 ns (3 allocations: 240 bytes)
Zonotope{Float64,SArray{Tuple{2},Float64,1,2},Array{Float64,2}}([-0.27985088856998697, -1.2396901825569322], [1.1510259645852767 0.0; 0.0 0.8583562373534231])
```

In [ ]:
convert(Zonotope, b).generators

----

## Higher dimensions

In [1]:
using Revise, LazySets, StaticArrays, BenchmarkTools, Test
using LazySets: isapproxzero

┌ Info: Precompiling LazySets [b4f0291d-fe17-52bc-9479-3d1a343d9043]
└ @ Base loading.jl:1278


In [4]:
b = BallInf([1.0, 2.0, 3.0], 2.0)
bs = BallInf(SA[1.0, 2.0, 3.0], 2.0);

bflat = BallInf([1.0, 2.0, 3.0], 0.0)
bsflat = BallInf([1.0, 2.0, 3.0], 0.0)

h = Hyperrectangle([1.0, 2.0, 3.0], [1.0, 2.0, 3.0])
hs = Hyperrectangle(SA[1.0, 2.0, 3.0], SA[1.0, 2.0, 3.0])

hflat = Hyperrectangle([1.0, 2.0, 3.0], [1.0, 0.0, 3.0])
hsflat = Hyperrectangle(SA[1.0, 2.0, 3.0], SA[1.0, 0.0, 3.0])

println("ballinf")
@btime convert(Zonotope, $b)
@btime convert(Zonotope, $bs)
@btime convert(Zonotope, $bflat)
@btime convert(Zonotope, $bsflat)

println("hyper")
@btime convert(Zonotope, $h)
@btime convert(Zonotope, $hs)
@btime convert(Zonotope, $hflat)
@btime convert(Zonotope, $hsflat)

println("hyper-no prune")
@btime LazySets._convert_static(Zonotope, $hs)
@btime LazySets._convert_static(Zonotope, $hsflat)

ballinf
  116.906 ns (3 allocations: 288 bytes)
  85.217 ns (3 allocations: 224 bytes)
  76.655 ns (3 allocations: 208 bytes)
  76.614 ns (3 allocations: 208 bytes)
hyper
  118.475 ns (3 allocations: 288 bytes)
  1.618 μs (28 allocations: 1.69 KiB)
  106.797 ns (3 allocations: 256 bytes)
  1.706 μs (28 allocations: 1.64 KiB)
hyper-no prune
  2.455 ns (0 allocations: 0 bytes)
  2.455 ns (0 allocations: 0 bytes)


Zonotope{Float64,SArray{Tuple{3},Float64,1,3},SArray{Tuple{3,3},Float64,2,9}}([1.0, 2.0, 3.0], [1.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 3.0])

In [3]:
LazySets._convert_static

_convert_static (generic function with 1 method)

In [ ]:
typeof(b)

In [ ]:
# problem: very slow 
#   2.195 μs (28 allocations: 1.75 KiB)
#=
function _genmat(B::BallInf{N, SVector{L, N}}) where {L, N}
    G = genmat(B)
    n, p = size(G)
    return SMatrix{n, p, N, n*p}(G)
end
=#

function _genmat(B::BallInf{N, SVector{L, N}}) where {L, N}
    r = B.radius
    if isapproxzero(B.radius)
        return SMatrix{L, 0, N, 0}()
    else
        gens = zeros(MMatrix{L, L})
        @inbounds for i in 1:L
            gens[i, i] = r
        end
        return SMatrix(gens)
    end
end

In [ ]:
b0 = BallInf(SA[1.0, 2.0, 3.0], 0.0)
b1 = BallInf(SA[1.0, 2.0, 3.0], 2.0)

@btime _genmat($b0)
@btime _genmat($b1)

In [ ]:
b0 = BallInf(SA[1.0, 2.0], 0.0)
b1 = BallInf(SA[1.0, 2.0], 2.0)

@btime _genmat($b0)
@btime _genmat($b1)

In [ ]:
function _genmat(H::Hyperrectangle{N, SVector{L, N}, SVector{L, N}}) where {L, N}   
    gens = zeros(MMatrix{L, L})
    @inbounds for i in 1:L
        r = H.radius[i]
        if !isapproxzero(r)
            gens[i, i] = r
        end
    end
    return SMatrix(gens)
end

function _new_convert()

In [ ]:
h0 = Hyperrectangle(SA[1.0, 2.0, 3.0], SA[4.0, 5.0, 6.0])
h1 = Hyperrectangle(SA[1.0, 2.0, 3.0], SA[4.0, 0.0, 1.0])

@btime _genmat($h0)
@btime _genmat($h1)

In [ ]:
h0 = Hyperrectangle(SA[1.0, 2.0], SA[4.0, 5.0])
h1 = Hyperrectangle(SA[1.0, 2.0], SA[4.0, 0.0])

@btime _genmat($h0)
@btime _genmat($h1)

In [ ]:
gens = generators(a)
@which LazySets.genmat_fallback(a, gens=gens, ngens=length(gens))

In [ ]:
gb = generators(b)
eltype(gb)

In [ ]:
@btime Zonotope(center($a), genmat($a))

In [ ]:
@which genmat(a)

In [ ]:

@btime convert(Zonotope, $a)
@btime convert(Zonotope, $b)
@btime LazySets._convert_2D_static(Zonotope, $b)

h = rand(Hyperrectangle, dim=3)
hs = Hyperrectangle(SVector{3,Float64}(h.center), SVector{3,Float64}(h.radius))

@btime convert(Zonotope, $h)
@btime convert(Zonotope, $hs)
@btime LazySets._convert_2D_static(Zonotope, $hs)